<a href="https://colab.research.google.com/github/whopriyam/Benchmarking-Transformer-Networks-for-Sexual-Harassment-Detection-on-Twitter/blob/main/Models/Electra_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers
# Optional: for Experiment management
!pip install wandb

     |████████████████████████████████| 225kB 31.4MB/s 
     |████████████████████████████████| 235kB 48.4MB/s 
     |████████████████████████████████| 2.3MB 46.4MB/s 
     |████████████████████████████████| 1.8MB 36.9MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
     |████████████████████████████████| 3.3MB 53.3MB/s 
     |████████████████████████████████| 8.2MB 47.1MB/s 
     |████████████████████████████████| 51kB 9.3MB/s 
     |████████████████████████████████| 122kB 53.5MB/s 
     |████████████████████████████████| 1.2MB 52.0MB/s 
     |████████████████████████████████| 245kB 51.0MB/s 
     |████████████████████████████████| 112kB 54.6MB/s 
     |████████████████████████████████| 901kB 48.7MB/s 
     |████████████████████████████████| 174kB 49.5MB/s 
     |████████████████████████████████| 133kB 56.2MB/s 
     |████████████████████████████████| 102kB 14.0MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 4.2MB 45.5MB

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/text classification/Harassment_Cleaned_tweets.csv')

In [4]:
df = df.filter(['Text','Label'], axis=1)
df

,Text,Label
0,He’d have my phone wedged up his ass sideways.,1
1,Yep 100% agree and the same with severine in s...,0
2,Stop having sex with men that won’t eat your p...,1
3,"When you see a sad man, just give him breast t...",1
4,We can't be together if you don't eat pussy,1
...,...,...
3599,only for a 5'7 brown hair brown eye gamer girl...,1
3600,"Don't be suspicious! Some butt, some suggestiv...",0
3601,mhmmmmm i hear ya big tittie lil booty,1
3602,I hate dirty stank ass cowboy fans bro lmao,0


In [5]:
train_df = df.sample(frac=0.7)
test_df = df.loc[~df.index.isin(train_df.index)]


train_final = train_df.sample(frac = 0.8)
eval_df = train_df.loc[~train_df.index.isin(train_final.index)]
len(eval_df), len(train_df) , len(test_df)

(505, 2523, 1081)

In [6]:
import pandas as pd
import pytest

from simpletransformers.classification import (
    ClassificationModel,
    # MultiLabelClassificationModel,
)


model_type = "electra"
model_name = "google/electra-small-discriminator"

In [8]:
 model = ClassificationModel(
        model_type,
        model_name,
        use_cuda=False,
        args={
            "no_save": True,
            "reprocess_input_data": True,
            "overwrite_output_dir": True,
            "scheduler": "constant_schedule",
            "max_seq_length": 20,
        },
    )

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/54.2M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['pooler.den

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [9]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:602: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/2523 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/316 [00:00<?, ?it/s]

(316, 0.6417675158834155)

In [11]:
len(test_df)

1081

In [10]:
# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_df)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1384: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1081 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/136 [00:00<?, ?it/s]

In [12]:
list_eval = eval_df['Text'].to_list()
type(list_eval)

list

In [13]:
predictions, raw_outputs = model.predict(list_eval)

  0%|          | 0/505 [00:00<?, ?it/s]

  0%|          | 0/64 [00:00<?, ?it/s]

In [14]:
from sklearn import metrics


In [15]:
targets = eval_df['Label'].to_list()
metrics.accuracy_score(targets, predictions)

0.7326732673267327

In [16]:
print(metrics.classification_report(targets, predictions))

              precision    recall  f1-score   support

           0       0.73      0.62      0.67       222
           1       0.73      0.82      0.78       283

    accuracy                           0.73       505
   macro avg       0.73      0.72      0.72       505
weighted avg       0.73      0.73      0.73       505

